In [13]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import model_selection, linear_model, metrics
from joblib import load

In [14]:
# чтение данных: валидационная выборка
df = pd.read_csv('val.tsv',sep='\t')
libs = df.libs
df.head()

,is_virus,filename,libs
0,1,2020-02-17T21_09_26+00_00_107385.exe,"kernel32.dll,advapi32.dll,comctl32.dll,gdi32.d..."
1,0,tac.exe,"msys-intl-8.dll,msys-2.0.dll,kernel32.dll"
2,1,VirusShare_eabeaf414d1b87659870918dc7f03f8a,kernel32.dll
3,1,VirusShare_f528efa5b5ba6643b7eb9cf4264d37e4,kernel32.dll
4,1,2019-08-14T00_52_23+00_00_60898.exe,msvbvm60.dll


In [15]:
# размер валидационной выборки
df.shape

(1200, 3)


In [16]:
# наличие пропусков в данных
df.isnull().values.any()

False

In [22]:
# предобработка данных
# преобразуем исходные данные в матрицу

df_train = pd.read_csv('train.tsv',sep='\t')
libs_train = df_train.libs
vectorizer = CountVectorizer(tokenizer = lambda x: x.split(','))
vectorizer.fit_transform(libs_train)

V = vectorizer.transform(libs)
X_val = V.toarray()
y_val = df['is_virus'].to_numpy()

In [7]:
# оценка качества модели на валидационной выборке
classifier = load('SGD_Model.joblib')
prediction_val = classifier.predict(X_val)
metrics.classification_report(y_val, prediction_val)

              precision    recall  f1-score   support

           0       0.91      0.69      0.78       400
           1       0.86      0.97      0.91       800

    accuracy                           0.87      1200
   macro avg       0.89      0.83      0.85      1200
weighted avg       0.88      0.87      0.87      1200



In [8]:
# вывод матрицы ошибок валидационной выборки
metrics.confusion_matrix(y_val, prediction_val)

[[275 125]
 [ 26 774]]


In [9]:
tn, fp, fn, tp = metrics.confusion_matrix(y_val, classifier.predict(X_val)).ravel()
accuracy = metrics.accuracy_score(y_val, classifier.predict(X_val))
precision = metrics.precision_score(y_val, classifier.predict(X_val))
recall = metrics.recall_score(y_val, classifier.predict(X_val))
f1 = metrics.f1_score(y_val, classifier.predict(X_val))

In [10]:
lines = []
lines.append('True positive: {}'.format(tp))
lines.append('False positive {}'.format(fp))
lines.append('False negative: {}'.format(fn))
lines.append('True negative: {}'.format(tn))
lines.append('Accuracy: {:.4}'.format(accuracy))
lines.append('Precision: {:.4}'.format(precision))
lines.append('Recall: {:.4}'.format(recall))
lines.append('F1: {:.4}'.format(f1))

In [11]:
with open('validation.txt','w') as file:
    file.writelines(line + '\n' for line in lines)